In [13]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [14]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe' 

In [15]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [25]:
def donga(query,driver_path, page):
    domain = '동아일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    
    current_page =1
    last_page =(int(page)-1)*10+1
    
    data_list =[]
    head_list=[]
    
    while current_page < last_page:
        print(current_page)
        driver =webdriver.Chrome(executable_path =driver_path, options =options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1020&nso=&is_sug_officeid=0'
        url = url_domain + query + '&%2Ca%3A&start='+str(current_page) + news_domain
        print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
 
    # 기사 url을 list형태로 반환
        try:
            
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
    
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
            
        except:
            print('기사가 없습니다.')
            driver.close()
            return ()
        driver.close()
    
    
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            try:
            
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
        # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#container > div.article_title > h1').text)
                tags = soup.select_one('#content > div > div.article_txt').text
                tags=clean_text(tags)
                data_list.append(tags)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page += 10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

dong = donga('풀무원 "바른먹거리"',driver_path,2)

1
https://search.naver.com/search.naver?where=news&query=풀무원 "바른먹거리"&%2Ca%3A&start=1&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1020&nso=&is_sug_officeid=0
http://economy.donga.com/3/all/20160426/77793584/2
None_type
http://bizn.donga.com/3/all/20180201/88471267/2
None_type
http://bizn.donga.com/3/all/20170711/85298938/2
None_type


In [28]:
dong.to_csv('동아일보.csv',encoding='utf-8-sig',index=None)
test=pd.read_csv('동아일보.csv',encoding='utf-8-sig')

In [29]:
test

,언론사,제목,내용
0,동아일보,풀무원 ‘식중독 케이크’ 논란에 여론 격앙…“친환경·바른먹거리 라며?”,사진식품의약품안전처집단 식중독 증세를 일으킨 원인으로 의심받고 있는 케이크 ‘우리밀...
1,동아일보,"[CSV 포터 상]풀무원, CSV는 아이 올바른 먹거리 챙기기부터!",‘바른먹거리 캠페인’ 교육을 받고 있는 초등학생들 풀무원 제공현재 풀무원의 대표적인...
2,동아일보,[Food&Dining3.0]‘바른먹거리 캠페인’ 교육 1만 명 실시,풀무원 풀무원이 대표적인 공유가치창출 활동인 ‘바른먹거리 캠페인’ 교육을 확대 실시...
3,동아일보,“두부 응용식품 ‘하프앤하프’ 뜨거운 반응… 시장 규모 3년 내 1000억 원까지 ...,■ 이효율 풀무원식품 사장 육가공 대체메뉴 직접 챙겨 “두부 응용식품 시장은 앞으로...
4,동아일보,"풀무원다논, 국제낙농연맹 연차총회서 ‘요거트 시장 성장과 혁신’ 발표",16일 대전 컨벤션 센터에서 열린 ‘2018 국제낙농연맹 연차 총회’에서 풀무원다논...
5,동아일보,"풀무원다논 그릭, 2년 연속 두 자릿수 성장",소비자가 ‘풀무원 다논 그릭 프로즌 딸기맛’을 고르고 있다 풀무원다논 그릭은 무더운...
6,동아일보,"[기업 브리핑]LG전자 ‘이제, 엄마의 …’ 이벤트 外",■ 전자 ‘이제 엄마의 …’ 이벤트전자가 고객들의 사연을 받아 양문형 냉장고 ‘디오...
